In [1]:
from loadfile import *
from phonemes import *
from preprocess import *
from evaluate import *
from google_kb_z_speaker.main import *
from tqdm.auto import tqdm
from experiment_helpers import *

2022-03-07 11:30:19,438.438 DEBUG __init__:  matplotlib data path: /home/nsmy/miniconda3/envs/speechTech/lib/python3.8/site-packages/matplotlib/mpl-data
2022-03-07 11:30:19,440.440 DEBUG __init__:  CONFIGDIR=/home/nsmy/.config/matplotlib
2022-03-07 11:30:19,442.442 DEBUG __init__:  interactive is False
2022-03-07 11:30:19,442.442 DEBUG __init__:  platform is linux
2022-03-07 11:30:19,442.442 DEBUG __init__:  loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_warnings', '_io', 'marshal', 'posix', '_frozen_importlib_external', '_thread', '_weakref', 'time', 'zipimport', '_codecs', 'codecs', 'encodings.aliases', 'encodings', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', '_abc', 'abc', 'io', '_stat', 'stat', '_collections_abc', 'genericpath', 'posixpath', 'os.path', 'os', '_sitebuiltins', '_locale', '_bootlocale', 'types', 'importlib._bootstrap', 'importlib._bootstrap_external', 'warnings', 'importlib', 'importlib.machinery', 'importlib.abc', '_operator', '

# Text processing

In [38]:
# Speaker
speakers = ["z21", "z20", "z28", "z29"]
models = ["google", "correct", "kb"]
models2fnameendings = {
    "google": "googleasr",
    "correct": "correct",
    "kb": "kb"
}
use_new = False

# Create filenames model -> List[List[str]] (List[str] are the lines from one filename
fnames = {model: [f"transcriptions{'_new' if use_new else ''}/{sp}.{models2fnameendings[model]}" for sp in speakers]
          for model in models}

bunches = {model: [get_fname_lines(fname) for fname in fnames] for model, fnames in fnames.items()}

# speakerLines : List[str]
# bunch List[speakerLines]
# bunches Dict[model, List[speakerLines]]

# Create a list [Dict[model,speakerlines] for speaker]

list_of_model2speaker_lines = [{model: bunch[i] for model, bunch in bunches.items()} for i in range(len(speakers))]
[fix_lines(x) for x in list_of_model2speaker_lines]
# recreate bunches Dict[model, List[speakerLines]]
bunches = {x: [list_of_model2speaker_lines[i][x] for i in range(len(speakers))] for x in models}
# Reduce bunches to bunches_reduced Dict[model, speakerLines_aggr]
bunches = {k: reduce(lambda x, y: x + y, v) for k, v in bunches.items()}
# Extract the transcriptions
bunches = {k: transcriptions(v) for k, v in bunches.items()}
# Preprocess  each transcription
bunches = {k: [preprocess_text(x) for x in v] for k, v in bunches.items()}

# Initialize phonemizer
phonemizer = init_phonemizer("cuda", "./models/deep-phonemizer-se.pt")

In [39]:
from IPython.core.debugger import set_trace

In [47]:
def singular_phonemes(txt : str):
    """
    Creates a list of singular phonemes from the
    output of get_swedish_phonemes
    """
    return txt.replace("_", " ").split()

def singular_phonemes_preprocess(bunches):
    return {k: [" ".join(singular_phonemes(s)) for s in v] for k,v in bunches.items()}

def get_swedish_phonemes_z(bunches, phonemizer):
    # Phonemizer creates a dict of words2phonemes for each line, so it 
    # so all phonemes should be created with a single call for consistency
    # Impose an order to models
    models = list(bunches.keys())
    num_lines = len(bunches["google"])
    concatenated = reduce(lambda x,y : x+y,[bunches[x] for x in models])
    phoneme_lines = get_swedish_phonemes(concatenated, phonemizer)
    return { model : phoneme_lines[num_lines * i:num_lines * (i+1)] for model,i in zip(models,range(0,len(models)))}

# Experiments

In [48]:
def evaluate_lines(bunches, _filter=None, phoneme_words=False, singular_phonemes=False, preprocess_hook=None): 
    if phoneme_words:
        bunches = get_swedish_phonemes_z(bunches, phonemizer)
        #
        bunches = {k : [preprocess_phonemes(x) for x in v] for k,v in bunches.items()}
              
    if singular_phonemes:
        bunches = get_swedish_phonemes_z(bunches, phonemizer)
        bunches = singular_phonemes_preprocess(bunches)
    
    google_lines = bunches["google"]
    correct_lines = bunches["correct"]
    kb_lines = bunches["kb"]

    if preprocess_hook is not None:
        preprocess_hook(**locals())

    if _filter is not None:
        if len(_filter) == 1 and "agreement" in _filter:
            correct_lines, google_lines, kb_lines = filter_lines_only_on_agreement(correct_lines, google_lines,
                                                                                kb_lines, _filter["agreement"])
        else:
            correct_lines, google_lines, kb_lines = filter_lines(correct_lines,google_lines,
                                                               kb_lines,**_filter)
    
    kei = 0
    oei = 0
    gei = 0
    tei = 0

    for c_l, g_l, kb_l in zip(correct_lines, google_lines, kb_lines):
        g_s = set(error_idxs(c_l, g_l))
        k_s = set(error_idxs(c_l, kb_l))

        gei += len(g_s)
        kei += len(k_s)
        oei += len(g_s & k_s)
        tei += len(g_s | k_s)

    error_index_overlap = oei / tei
   
    agreement, g_correct_kb_not, kb_correct_g_not, agreement_not_correct, agreement_correct,\
    both_incorrect_disagreement =\
        percentage_of_agreement(correct_lines, google_lines, kb_lines)

    
    
    return {
        "google_wer" : wer(correct_lines, google_lines),
        "kb_wer" : wer(correct_lines, kb_lines),
        "agreement" : agreement,
        "g_correct_kb_not " : g_correct_kb_not,
        "kb_correct_g_not " : kb_correct_g_not,
        "agreement_not_correct" :  agreement_not_correct,
        "both_incorrect_disagreement" : both_incorrect_disagreement,
        "error_index_overlap": error_index_overlap
    }

In [49]:
lines = []
def locate_empty_lines(**kwargs):
    print(kwargs["google_lines"])
        

In [50]:
experiment = evaluate_lines
kwargs = {
    "_filter": None, 
    "phoneme_words" : False,
    "singular_phonemes" : True,
    "preprocess_hook" : None
}
res = experiment_repeats(experiment, 5, bunches, **kwargs)
print_experiment_report(res)

  0%|          | 0/5 [00:00<?, ?it/s]

google_wer: 0.12073146586826265±0.0002185139381326077
kb_wer: 0.08920142950341382±0.00043211676094552634
agreement: 0.3729967069154775±0.0004909040565312246
g_correct_kb_not : 0.2804618323882772±0.0005627618940837472
kb_correct_g_not : 0.1684181842675713±0.002353003642009457
agreement_not_correct: 0.09123026201631095±0.00012028128978395692
both_incorrect_disagreement: 0.5511199833441515±0.0020939522601463724
error_index_overlap: 0.20093572065942±0.0010317631106002286


In [176]:
google_lines.index("30")

238

In [177]:
set(phonemizer(["jag" for x in range(10000)], "se"))

{"j'a2:"}

In [ ]:
set(get_swedish_phonemes(["ja" for x in range(10000)], phonemizer))

In [ ]:
s = []
for x in tqdm(range(1000)):
    torch.manual_seed(0)
    s.append(phonemizer('30', "se"))
set(s)

In [ ]:
s = []
for x in tqdm(range(15)):
    #torch.manual_seed(0)
    s.append(get_swedish_phonemes('ja', phonemizer))
set(s)

In [154]:
idx = 43
size = 10
def get_triad(idx, bunch):
    #### Google, correct, Kb
    return bunch[0][idx], bunch[1][idx], bunch[2][idx]

def plot_triad(triad,ax):
    ax.text(0.0, 0.0, triad[0], size=size, rotation=0,
             ha="center", va="center",
             bbox=dict(boxstyle="round",
                       ec=(1., 0.5, 0.5),
                       fc=(1., 0.8, 0.8),
                       )
             )

    ax.text(0, -2.5,  triad[2], size=size, rotation=0,
             ha="center", va="center",
             bbox=dict(boxstyle="round",
                       ec=(153/255, 51/255, 0/255),
                       fc=(255/255, 153/255, 102/255),
                       )
             )
    ax.text(0, -5,  triad[1], size=size, rotation=0,
             ha="center", va="center",
             bbox=dict(boxstyle="round",
                       ec=(42 / 255, 162 / 255, 42 / 255),
                       fc=(133 / 255, 224 / 255, 133 / 255),
                       )
             )
    
    ax.set_ylim(-10, 10)
    ax.set_xlim(-10, 10)

SyntaxError: invalid syntax (3878495266.py, line 2)

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(15, 5))
idx = 0
plot_triad(get_triad(idx, text_data_bunch) ,axs[0])
plot_triad(get_triad(idx, phoneme_data_bunch) ,axs[1])

In [ ]:

def print_triad_correctness(triad):
    #### Google, correct, Kb
    print("Google correct ", triad[1] == triad[0])
    print("KB correct ", triad[1] == triad[2])
    print("Agreement ", triad[0] == triad[2])
    
    print("Google WER ", wer(triad[1], triad[0]))
    print("KB WER ", wer(triad[1], triad[2]))